# **Hotel Recommendation System with NLP**

In [18]:
!pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from ast import literal_eval

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
data = pd.read_csv("Hotel_Reviews.csv")
data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194.0,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397.0,1403.0,Only the park outside of the hotel was beauti...,11.0,7.0,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194.0,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0.0,1403.0,No real complaints the hotel was great great ...,105.0,7.0,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194.0,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42.0,1403.0,Location was good and staff were ok It is cut...,21.0,9.0,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194.0,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210.0,1403.0,Great location in nice surroundings the bar a...,26.0,1.0,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194.0,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140.0,1403.0,Amazing location and building Romantic setting,8.0,3.0,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [5]:
data.Hotel_Address = data.Hotel_Address.str.replace("United Kingdom","UK")
data["countries"] = data.Hotel_Address.apply(lambda x: x.split(' ')[-1])
print(data.countries.unique())

['Netherlands' 'UK' 'France' 'Greenwi']


In [7]:
data.drop(['Additional_Number_of_Scoring',
           'Review_Date', 'Reviewer_Nationality',
           'Negative_Review', 'Review_Total_Negative_Word_Counts',
           'Total_Number_of_Reviews', 'Positive_Review',
           'Review_Total_Positive_Word_Counts',
           'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
           'days_since_review', 'lat', 'lng'], axis=1, inplace=True)

In [11]:
def impute(column):
  column = column[0]
  if isinstance(column, str) and not pd.isna(column):
    try:
        return "".join(literal_eval(column))
    except (SyntaxError, ValueError):
        return column

  else:
    return column

data["Tags"] = data[["Tags"]].apply(impute, axis=1)
data.head()

<ipython-input-11-66bfa5bef953>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  column = column[0]


,Hotel_Address,Average_Score,Hotel_Name,Tags,countires,countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Sta...,Netherlands,Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Sta...,Netherlands,Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Family with young children Dup...,Netherlands,Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Solo traveler Duplex Double Ro...,Netherlands,Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Suite Stayed 2 nights ...,Netherlands,Netherlands


In [12]:
data['countries'] = data['countires'].str.lower()
data['Tags'] = data['Tags'].str.lower()

In [13]:
def recommend_hotel(location, description):
  description = description.lower()
  word_tokenize(description)
  stop_words = stopwords.words('english')
  lemm = WordNetLemmatizer()
  filtered = {word for word in description if not word in stop_words}
  filtered_set = set()
  for fs in filtered:
    filtered_set.add(lemm.lemmatize(fs))
  country = data[data['countries']==location.lower()]
  country = country.set_index(np.arange(country.shape[0]))
  list1 = []; list2 = []; cos = [];
  for i in range(country.shape[0]):
    temp_token = word_tokenize(country["Tags"][i])
    temp_set = [word for word in temp_token if not word in stop_words]
    temp2_set = set()
    for s in temp_set:
      temp2_set.add(lemm.lemmatize(s))
    vector = temp2_set.intersection(filtered_set)
    cos.append(len(vector))

  country['similarity'] = cos
  country = country.sort_values(by = 'similarity', ascending = False)
  country.drop_duplicates(subset = 'Hotel_Name', keep = 'first', inplace= True)
  country.sort_values('Average_Score', ascending = False, inplace = True)
  country.reset_index(inplace=True)
  return country[["Hotel_Name", "Average_Score", "Hotel_Address"]].head()

# **Working**

In [20]:
recommend_hotel('UK', 'I am going to a business trip')

,Hotel_Name,Average_Score,Hotel_Address
0,Haymarket Hotel,9.6,1 Suffolk Place Westminster Borough London SW1...
1,Milestone Hotel Kensington,9.5,1 Kensington Court Kensington and Chelsea Lond...
2,Charlotte Street Hotel,9.5,15 17 Charlotte Street Hotel Westminster Borou...
3,Covent Garden Hotel,9.4,10 Monmouth Street Camden London WC2H 9HB UK
4,The Goring,9.4,15 Beeston Place Westminster Borough London SW...


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Average_Score'].plot(kind='hist', bins=20, title='Average_Score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('Hotel_Name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('Hotel_Address').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['Hotel_Address'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='index', y='Hotel_Address', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['Hotel_Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='Average_Score', y='Hotel_Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['Hotel_Address'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='Average_Score', y='Hotel_Address', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)